
# PaymanAI

PaymanAI provides functionality to send and receive payments (fiat and crypto) on behalf of an AI Agent. To get started:

1. **Sign up** at app.paymanai.com to create an AI Agent and obtain your **API Key**.
2. **Set** environment variables (`PAYMAN_API_SECRET` for your API Key, `PAYMAN_ENVIRONMENT` for sandbox or production).

This notebook gives a quick overview of integrating PaymanAI into LangChain as a tool. For complete reference, see the API documentation.

## Overview

The PaymanAI integration is part of the `langchain-community` (or your custom) package. It allows you to:

- Send payments (`send_payment`) to crypto addresses or bank accounts.
- Search for payees (`search_payees`).
- Add new payees (`add_payee`).
- Request money from customers with a hosted checkout link (`ask_for_money`).
- Check agent or customer balances (`get_balance`).

These can be wrapped as **LangChain Tools** for an LLM-based agent to call them automatically.

### Integration details

| Class | Package | Serializable | JS support | Package latest |
| :--- | :--- | :---: | :---: | :--- |
| PaymanAI | `langchain_community` | ❌ | ❌ | [PyPI Version] |

If you're simply calling the PaymanAI SDK, you can do it directly or via the **Tool** interface in LangChain.

## Setup

1. **Install** the `langchain-community` (or equivalent) package:

```bash
pip install --quiet -U langchain-community
```

2. **Install** the PaymanAI SDK:
```bash
pip install paymanai
```

3. **Set** environment variables:
```bash
export PAYMAN_API_SECRET="YOUR_SECRET_KEY"
export PAYMAN_ENVIRONMENT="sandbox"
```

Your `PAYMAN_API_SECRET` should be the secret key from app.paymanai.com. The `PAYMAN_ENVIRONMENT` can be `sandbox` or `production` depending on your usage.

## Instantiation

Here is an example of instantiating a PaymanAI tool. If you have multiple Payman methods, you can create multiple tools.

```python
from langchain_community.tools.langchain_payman_tool.tool import PaymanAI

# Instantiate the PaymanAI tool (example)
tool = PaymanAI(
    name="send_payment",
    description="Send a payment to a specified payee.",
)
```

## Invocation

### Invoke directly with args

You can call `tool.invoke(...)` and pass a dictionary matching the tool's expected fields. For example:

```python
response = tool.invoke({
    "amount_decimal": 10.00,
    "payment_destination_id": "abc123",
    "customer_id": "cust_001",
    "memo": "Payment for invoice #XYZ"
})
```

### Invoke with ToolCall

When used inside an AI workflow, the LLM might produce a `ToolCall` dict. You can simulate it as follows:

```python
model_generated_tool_call = {
    "args": {
        "amount_decimal": 10.00,
        "payment_destination_id": "abc123"
    },
    "id": "1",
    "name": tool.name,
    "type": "tool_call",
}
tool.invoke(model_generated_tool_call)
```

## Using the Tool in a Chain or Agent

You can bind a PaymanAI tool to a LangChain agent or chain that supports tool-calling.

## Quick Start Summary

1. **Sign up** at app.paymanai.com to get your **API Key**.
2. **Install** dependencies:
   ```bash
   pip install paymanai langchain-community
   ```
3. **Export** environment variables:
   ```bash
   export PAYMAN_API_SECRET="YOUR_SECRET_KEY"
   export PAYMAN_ENVIRONMENT="sandbox"
   ```
4. **Instantiate** a PaymanAI tool, passing your desired name/description.
5. **Call** the tool with `.invoke(...)` or integrate it into a chain or agent.

## API reference

You can find full API documentation for PaymanAI at:

- [Langchain-Payman Python reference](https://pypi.org/project/langchain-payman-tool/)
- [Payman Docs](https://docs.paymanai.com)

## Chaining

```python
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain.chat_models import init_chat_model

# Assume we've imported your PaymanAITool or multiple Payman AI Tools
payman_tool = PaymanAITool(name="send_payment")

# Build a prompt
prompt = ChatPromptTemplate([
    ("system", "You are a helpful AI that can send payments if asked."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

llm = init_chat_model(model="gpt-4", model_provider="openai")
llm_with_tools = llm.bind_tools([payman_tool], tool_choice=payman_tool.name)

llm_chain = prompt | llm_with_tools

@chain
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    tool_msgs = payman_tool.batch(ai_msg.tool_calls, config=config)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# Example usage:
response = tool_chain.invoke("Send $10 to payee123.")
print(response)```

## API reference

You can find full API documentation for PaymanAI at:

- [Python reference](https://python.langchain.com/v0.2/api_reference/community/tools/langchain_community.tools.langchain_payman_tool.tool.PaymanAI.html)
- (Any other relevant references or doc links)